## setup libraries

In [2]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

!pip install gdelt

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://security.ubuntu.c

In [3]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

## fetch gdelt data for a given date range

Note: this example only fetches the events table. The gdelt python API also can pull from the mentions and gkg tables. See https://linwoodc3.github.io/gdeltPyR/ for full API details



In [4]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()


# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)

def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date, table='events',coverage=False) #not updata at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
    except:
        print("Error occurred")

# 测试下载两天的event table
test_data = [get_filename(x) for x in pd.date_range('2021 Nov 1','2021 Nov 2')]
dbg(test_data)
#开始下载
results = list(e.map(intofile,test_data))

['20211101_gdeltdata.csv', '20211102_gdeltdata.csv']


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = eventWork(self.download_list)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = eventWork(self.download_list)


## read downloaded files into RDDs

In [5]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

test_data1 = sqlContext.read.option("header", "true").csv(test_data)
#dbg(test_data)
test_data_rdd = test_data1.rdd
dbg(test_data_rdd)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


[('1012211458', '20201101'), ('1012211459', '20201101'), ('1012211460', '20201101'), ('1012211461', '20201101'), ('1012211462', '20201101'), ('1012211463', '20201101'), ('1012211464', '20201101'), ('1012211465', '20201101'), ('1012211466', '20211002'), ('1012211467', '20211002'), ('1012211468', '20211002'), ('1012211469', '20211002'), ('1012211470', '20211002'), ('1012211471', '20211025'), ('1012211472', '20211025'), ('1012211473', '20211025'), ('1012211474', '20211025'), ('1012211475', '20211025'), ('1012211476', '20211025'), ('1012211477', '20211025'), ('1012211478', '20211025'), ('1012211479', '20211025'), ('1012211480', '20211025'), ('1012211481', '20211025'), ('1012211482', '20211025'), ('1012211483', '20211025'), ('1012211484', '20211025'), ('1012211485', '20211025'), ('1012211486', '20211025'), ('1012211487', '20211025'), ('1012211488', '20211025'), ('1012211489', '20211025'), ('1012211490', '20211031'), ('1012211491', '20211101'), ('1012211492', '20211101'), ('1012211493', '202

In [11]:
#去除了null的国家
countryName_And_Scale1 = test_data_rdd.filter(lambda x: x.Actor1CountryCode != None and x.Actor2CountryCode != None and x.Actor1CountryCode != x.Actor2CountryCode)
dbg(countryName_And_Scale1.collect())

[Row(GLOBALEVENTID='1012211467', SQLDATE='20211002', MonthYear='202110', Year='2021', FractionDate='2021.7452', Actor1Code='USA', Actor1Name='SAN ANTONIO', Actor1CountryCode='USA', Actor1KnownGroupCode=None, Actor1EthnicCode=None, Actor1Religion1Code=None, Actor1Religion2Code=None, Actor1Type1Code=None, Actor1Type2Code=None, Actor1Type3Code=None, Actor2Code='ISR', Actor2Name='ISRAEL', Actor2CountryCode='ISR', Actor2KnownGroupCode=None, Actor2EthnicCode=None, Actor2Religion1Code=None, Actor2Religion2Code=None, Actor2Type1Code=None, Actor2Type2Code=None, Actor2Type3Code=None, IsRootEvent='0', EventCode='140', CAMEOCodeDescription=' Engage in popular protest, not specified below', EventBaseCode='140', EventRootCode='14', QuadClass='3', GoldsteinScale='-6.5', NumMentions='1', NumSources='1', NumArticles='1', AvgTone='-3.85185185185185', Actor1Geo_Type='3', Actor1Geo_FullName='San Marcos, Texas, United States', Actor1Geo_CountryCode='US', Actor1Geo_ADM1Code='USTX', Actor1Geo_ADM2Code='TX209

In [12]:
#make two country as a key
countryName_And_Scale2 = countryName_And_Scale1.map(lambda a: (tuple((sorted((str(a.Actor1CountryCode), str(a.Actor2CountryCode))))), a.GoldsteinScale, a.AvgTone, a.NumMentions))
dbg(countryName_And_Scale2)

[(('ISR', 'USA'), '-6.5'), (('ISR', 'USA'), '-6.5'), (('AFG', 'GBR'), '3.4'), (('AFG', 'PAK'), '5.0'), (('AFG', 'PAK'), '5.0'), (('AUS', 'FRA'), '-4.0'), (('AUS', 'FRA'), '-4.0'), (('AUS', 'FRA'), '-4.0'), (('AUS', 'FRA'), '-4.0'), (('AUS', 'PHL'), '3.4'), (('AUS', 'USA'), '4.0'), (('AUS', 'USA'), '1.0'), (('BOL', 'BRA'), '1.9'), (('BOL', 'BRA'), '1.9'), (('BOL', 'BRA'), '-5.0'), (('BOL', 'BRA'), '2.8'), (('BRA', 'USA'), '4.0'), (('CAN', 'GBR'), '4.0'), (('CAN', 'GBR'), '4.0'), (('CAN', 'PHL'), '3.4'), (('CHE', 'USA'), '1.0'), (('CHE', 'USA'), '-2.0'), (('CHN', 'MEX'), '1.9'), (('CHN', 'PHL'), '1.9'), (('CHN', 'VNM'), '1.9'), (('DEU', 'GBR'), '7.0'), (('DOM', 'USA'), '2.8'), (('DOM', 'USA'), '2.8'), (('DOM', 'USA'), '2.8'), (('DOM', 'USA'), '2.8'), (('DOM', 'USA'), '2.8'), (('ESP', 'USA'), '3.0'), (('ETH', 'USA'), '0.4'), (('EUR', 'USA'), '2.8'), (('AUS', 'FRA'), '3.0'), (('AUS', 'FRA'), '3.0'), (('AUS', 'FRA'), '4.0'), (('FRA', 'GBR'), '3.4'), (('FRA', 'ITA'), '-2.0'), (('FRA', 'ITA')

In [10]:
#没用的东西
#country_And_Scale_broad = sc.broadcast(countryName_And_Scale2.collect())
#dbg(country_And_Scale_broad)


###  ***Meaning of the graph***
```
# countryName_And_Scale2 =(tuple((sorted((str(a.Actor1CountryCode), str(a.Actor2CountryCode))))), a.GoldsteinScale, a.AvgTone, a.NumMentions)
```
* [0]:key:(country1, counrty2);
* [1]:GoldsteinScale;
* [2]:AvgTone;
* [3]:NumMentions










In [17]:
#找频繁项集
country_And_Mentions = countryName_And_Scale2.flatMap(lambda a: [(i, int(a[3])) for i in a[0]])
country_And_Mentions_add = country_And_Mentions.reduceByKey(lambda a,b: a+b)
country_And_Mentions_filter = country_And_Mentions_add.filter(lambda a: a[1] >= 30).collectAsMap()
#broadcast filtered countrys
country_And_Mentions_filtered_list = sc.broadcast(country_And_Mentions_filter)
list(country_And_Mentions_filtered_list.value)

['USA',
 'AFG',
 'AUS',
 'DEU',
 'ETH',
 'GTM',
 'GBR',
 'PAK',
 'FRA',
 'BOL',
 'BRA',
 'CHN',
 'MEX',
 'EUR',
 'ITA',
 'PRK',
 'RUS',
 'YEM']

In [1]:
#寻找mention都大于1000次的pair/   Find the pair whose mention are greater than 1000 times
def select(event):
  if event[0][0] in country_And_Mentions_filtered_list.value and event[0][1] in country_And_Mentions_filtered_list.value:
    return event
  
frequent_set = countryName_And_Scale2.filter(lambda a: select(a))
dbg(countryName_And_Scale2)
dbg(frequent_set)

countryName_And_Mentions = frequent_set.map(lambda a: (a[0], int(a[3]))).reduceByKey(lambda a,b:a+b)
dbg(countryName_And_Mentions)


NameError: ignored